# 인구소멸 위기지역 - ID 부여 (Part 2)

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
pop = pd.read_csv('data/시군구_전처리완료.csv', encoding='euc-kr')
pop.head()

,광역시도,시군구,20-39세계,20-39세남자,20-39세여자,65세이상계,65세이상남자,65세이상여자,인구수계,인구수남자,인구수여자,소멸비율,소멸위기지역,소멸위기고위험지역
0,강원도,강릉시,47006,25199,21807,43830,18604,25227,211643,105025,106618,0.497536,True,False
1,강원도,고성군,5217,3175,2042,7755,3237,4519,26792,13864,12928,0.263314,True,False
2,강원도,동해시,18727,10469,8258,17572,7624,9949,89814,45572,44242,0.469952,True,False
3,강원도,삼척시,13583,7469,6117,16077,6766,9311,65623,33275,32348,0.380481,True,False
4,강원도,속초시,18080,9909,8171,15170,6345,8826,81497,40312,41185,0.538629,False,False


In [3]:
pop.shape

(250, 14)

- 고유 ID 만들기

In [8]:
np.unique(pop.시군구.values).shape

(226,)

In [5]:
np.unique(pop.시군구.values)

array(['가평군', '강남구', '강동구', '강릉시', '강북구', '강서구', '강진군', '강화군', '거제시',
       '거창군', '경산시', '경주시', '계룡시', '계양구', '고령군', '고성군', '고창군', '고흥군',
       '곡성군', '공주시', '과천시', '관악구', '광명시', '광산구', '광양시', '광주시', '광진구',
       '괴산군', '구례군', '구로구', '구리시', '구미시', '군산시', '군위군', '군포시', '권선구',
       '금산군', '금정구', '금천구', '기장군', '기흥구', '김제시', '김천시', '김포시', '김해시',
       '나주시', '남구', '남동구', '남양주시', '남원시', '남해군', '노원구', '논산시', '단양군',
       '단원구', '달서구', '달성군', '담양군', '당진시', '대덕구', '덕양구', '덕진구', '도봉구',
       '동구', '동남구', '동대문구', '동두천시', '동래구', '동안구', '동작구', '동해시', '마산합포구',
       '마산회원구', '마포구', '만안구', '목포시', '무안군', '무주군', '문경시', '미추홀구', '밀양시',
       '보령시', '보성군', '보은군', '봉화군', '부산진구', '부안군', '부여군', '부천시', '부평구',
       '북구', '분당구', '사상구', '사천시', '사하구', '산청군', '삼척시', '상당구', '상록구',
       '상주시', '서구', '서귀포시', '서대문구', '서북구', '서산시', '서원구', '서천군', '서초구',
       '성동구', '성북구', '성산구', '성주군', '세종시', '속초시', '송파구', '수성구', '수영구',
       '수정구', '수지구', '순창군', '순천시', '시흥시', '신안군', '아산시', '안동시', '안성시',
       '양구군',

In [9]:
# 고성군 - 고성(강원), 고성(경남)
# 광역시 - 서울 용산, 서울 서대문, 대전 서구, 대전 유성, 세종
# 행정구 - 수원 장안, 용인 수지, 고양 일산동, 창원 합포, 창원 회원
tmp_gu_dict = {
    '수원': ['장안구', '권선구', '팔달구', '영통구'], 
    '성남': ['수정구', '중원구', '분당구'], 
    '안양': ['만안구', '동안구'], 
    '안산': ['상록구', '단원구'], 
    '고양': ['덕양구', '일산동구', '일산서구'], 
    '용인': ['처인구', '기흥구', '수지구'], 
    '청주': ['상당구', '서원구', '흥덕구', '청원구'], 
    '천안': ['동남구', '서북구'], 
    '전주': ['완산구', '덕진구'], 
    '포항': ['남구', '북구'], 
    '창원': ['의창구', '성산구', '진해구', '마산합포구', '마산회원구']
}

In [10]:
metro_list = ['서울특별시','부산광역시','대구광역시','인천광역시','대전광역시','광주광역시','울산광역시']
si_name = [None] * len(pop)

for i in pop.index:
    if pop.광역시도[i] in metro_list:
        if len(pop.시군구[i]) == 2:
            si_name[i] = pop.광역시도[i][:2] + ' ' + pop.시군구[i]      # 서울 중구, 대전 서구
        else:
            si_name[i] = pop.광역시도[i][:2] + ' ' + pop.시군구[i][:-1] # 서울 용산, 서울 서대문
    else:
        if pop.시군구[i][:-1] == '고성':
            if pop.광역시도[i] == '강원도':
                si_name[i] = '고성(강원)'
            else:
                si_name[i] = '고성(경남)'
        else:
            si_name[i] = pop.시군구[i][:-1]         # 광명, 김포, 강릉

        for key, values in tmp_gu_dict.items():
            if pop.시군구[i] in values:
                if len(pop.시군구[i]) == 2:
                    si_name[i] = key + ' ' + pop.시군구[i]          # 포항 북구
                elif pop.시군구[i] in ['마산합포구', '마산회원구']:
                    si_name[i] = key + ' ' + pop.시군구[i][2:-1]    # 창원 합포, 창원 회원
                else:
                    si_name[i] = key + ' ' + pop.시군구[i][:-1]     # 용인 수지, 수원 팔달

In [11]:
print(si_name)

['강릉', '고성(강원)', '동해', '삼척', '속초', '양구', '양양', '영월', '원주', '인제', '정선', '철원', '춘천', '태백', '평창', '홍천', '화천', '횡성', '가평', '과천', '광명', '광주', '구리', '군포', '수원 권선', '용인 기흥', '김포', '남양주', '안산 단원', '고양 덕양', '동두천', '안양 동안', '안양 만안', '부천', '성남 분당', '안산 상록', '성남 수정', '용인 수지', '시흥', '안성', '양주', '양평', '여주', '연천', '수원 영통', '오산', '의왕', '의정부', '이천', '고양 일산동', '고양 일산서', '수원 장안', '성남 중원', '용인 처인', '파주', '수원 팔달', '평택', '포천', '하남', '화성', '거제', '거창', '고성(경남)', '김해', '남해', '창원 합포', '창원 회원', '밀양', '사천', '산청', '창원 성산', '양산', '의령', '창원 의창', '진주', '창원 진해', '창녕', '통영', '하동', '함안', '함양', '합천', '경산', '경주', '고령', '구미', '군위', '김천', '포항 남구', '문경', '봉화', '포항 북구', '상주', '성주', '안동', '영덕', '영양', '영주', '영천', '예천', '울릉', '울진', '의성', '청도', '청송', '칠곡', '광주 광산', '광주 남구', '광주 동구', '광주 북구', '광주 서구', '대구 남구', '대구 달서', '대구 달성', '대구 동구', '대구 북구', '대구 서구', '대구 수성', '대구 중구', '대전 대덕', '대전 동구', '대전 서구', '대전 유성', '대전 중구', '부산 강서', '부산 금정', '부산 기장', '부산 남구', '부산 동구', '부산 동래', '부산 부산진', '부산 북구', '부산 사상', '부산 사하', '부산 서구', '부산 수영', '부산 연제'

In [12]:
pop['ID'] = si_name
pop.drop(columns=['20-39세남자','65세이상남자','65세이상여자'], inplace=True)
print(pop.shape)
pop.head()

(250, 12)


,광역시도,시군구,20-39세계,20-39세여자,65세이상계,인구수계,인구수남자,인구수여자,소멸비율,소멸위기지역,소멸위기고위험지역,ID
0,강원도,강릉시,47006,21807,43830,211643,105025,106618,0.497536,True,False,강릉
1,강원도,고성군,5217,2042,7755,26792,13864,12928,0.263314,True,False,고성(강원)
2,강원도,동해시,18727,8258,17572,89814,45572,44242,0.469952,True,False,동해
3,강원도,삼척시,13583,6117,16077,65623,33275,32348,0.380481,True,False,삼척
4,강원도,속초시,18080,8171,15170,81497,40312,41185,0.538629,False,False,속초


In [13]:
pop.to_csv('data/시군구_ID부여.csv', index=False, encoding='euc-kr')